In [ ]:
using Revise
using FUSE
using Plots

In [ ]:
chk = FUSE.Checkpoint(); # checkpointing allows us to run mutliple times the same JupyterNotebook cell without everytime having to restart from scratch

ini,act = FUSE.case_parameters(:D3D;scenario=:L_mode,scenario_sources=false,scenario_core_profiles=false)
empty!(ini.general.dd.core_profiles)

n_before = 1.0e19
n_sep_rat_before = 0.7

ini.core_profiles.ne_sep_to_ped_ratio = n_sep_rat_before
ini.core_profiles.ne_value = n_before
ini.nb_unit[1].power_launched = 0.2e6
    
ini.core_profiles.zeff = 1.5
ini.core_profiles.ne_setting = :ne_ped
ini.core_profiles.Te_ped = 0.3e3
ini.core_profiles.Te_core = 1.5e3

ini.core_profiles.w_ped = 0.1

ini.time.simulation_start = 0.1

act.ActorPedestal.density_match=:ne_ped

dd = IMAS.dd()
FUSE.init(dd,ini,act);

for ion in dd.core_profiles.profiles_1d[].ion
    ion.temperature = dd.core_profiles.profiles_1d[].electrons.temperature
end

@checkin :hw_init dd ini act;
    
plot(dd.core_profiles;coordinate=:psi_norm)

In [ ]:
@checkout :hw_init dd ini act;


ini.time.pulse_shedule_time_basis = range(0, 1.2; step=0.1)

#ini.rampup.side = :lfs
#ini.rampup.ends_at = rampup_ends
#ini.rampup.diverted_at = rampup_ends * 0.8

#ini.equilibrium.pressure_core = t -> ramp(t / rampup_ends) .^ 2 * 2.22e6 .+ 5e5
#ini.equilibrium.ip = t -> ramp(t / rampup_ends) * 8.7e6 # + ramp((t - 7) / 20) * 1E6 # 7.7 + 1 ramp

ini.nb_unit[1].power_launched = t -> sequence(t, [(0.0, 1e5), (0.39, 1.0e5),(0.4, 2.5e6),  (0.89, 2.5e6), (0.9, 1e5)]);
#ini.ic_antenna[1].power_launched = t -> (3.1e6 .+ 8e6 .* ramp((t - 7) / 7)) # 11.1e6

ini.equilibrium.xpoints = :lower
# ...or by a `sequence(t, t_y_tuple_sequence)`
#ini.equilibrium.ip = t -> sequence(t, [(0.0, 0.0), (10.0, 13.0E6), (100.0, 13.0E6), (200.0, 15.0E6)]);

plot(ini)

In [ ]:
FUSE.init(dd, ini, act; initialize_hardware=false);

plot(dd.core_profiles)

@checkin :t_init dd ini act;

In [ ]:
FUSE.plot_plasma_overview(dd, dd.global_time; min_power=1E6, aggregate_radiation=true)

In [ ]:
@checkout :t_init dd ini act;

act.ActorStationaryPlasma.convergence_error = 2E-2
act.ActorStationaryPlasma.max_iter = 3

act.ActorCoreTransport.model = :FluxMatcher
act.ActorFluxMatcher.optimizer_algorithm = :simple
act.ActorFluxMatcher.max_iterations = 500
act.ActorFluxMatcher.verbose = false

act.ActorFluxMatcher.rho_transport = 0.2:0.05:0.9

act.ActorPedestal.model = :WPED
act.ActorWPED.ped_to_core_fraction =0.15

#act.ActorPedestal.rho_nml = 0.85
#act.ActorPedestal.rho_ped = 0.9
#act.ActorTGLF.tglfnn_model = "sat0quench_em_d3d+mastu_azf+1"


act.ActorFluxMatcher.evolve_pedestal = false
act.ActorEquilibrium.model = :TEQUILA

FUSE.ActorStationaryPlasma(dd, act; verbose=false)
@checkin :stationary dd ini act;
FUSE.plot_plasma_overview(dd)

In [ ]:
plot(dd.core_profiles)

In [ ]:
FUSE.plot_plasma_overview(dd)

In [ ]:
@checkout :stationary dd ini act;


act.ActorPedestal.model = :dynamic
act.ActorPedestal.ip_from = :pulse_schedule
act.ActorPedestal.zeff_ped_from = :core_profiles
act.ActorPedestal.ne_from = :pulse_schedule
act.ActorPedestal.ip_from = :pulse_schedule
act.ActorPedestal.βn_from = :equilibrium


act.ActorPedestal.rho_nml = act.ActorFluxMatcher.rho_transport[end-1]
act.ActorPedestal.rho_ped = act.ActorFluxMatcher.rho_transport[end]

act.ActorPedestal.tau_n = 0.3
act.ActorPedestal.tau_t = 0.2
act.ActorPedestal.density_ratio_L_over_H = 0.2 # 5 times multiplier of L to H

actor_ped = FUSE.ActorPedestal(dd,act.ActorPedestal,act)

actor_ped.state = [:L_mode]
actor_ped.cp1d_transition = deepcopy(dd.core_profiles.profiles_1d[])

act.ActorFluxMatcher.optimizer_algorithm = :simple
act.ActorFluxMatcher.rho_transport = 0.2:0.05:0.9


act.ActorDynamicPlasma.Δt = 1.0
act.ActorDynamicPlasma.Nt = 100

act.ActorDynamicPlasma.evolve_transport = true
act.ActorDynamicPlasma.evolve_current = false
act.ActorDynamicPlasma.evolve_equilibrium = false
act.ActorDynamicPlasma.evolve_pf_active = false
act.ActorDynamicPlasma.evolve_pedestal = true
act.ActorDynamicPlasma.evolve_hcd = true

act.ActorDynamicPlasma.verbose = true
act.ActorDynamicPlasma.time_derivatives_sources = false


actor_dyn = FUSE.ActorDynamicPlasma(dd,act.ActorDynamicPlasma,act)

actor_ped.cp1d_transition = deepcopy(dd.core_profiles.profiles_1d[])
actor_dyn.actor_ped = actor_ped


FUSE.finalize(FUSE.step(actor_dyn))

@checkin :time_dep dd ini act actor_dyn;

In [ ]:
plot(dd.summary.local.pedestal.n_e, :value)

In [ ]:

result = [s == :L_mode ? 0 : 1 for s in actor_dyn.actor_ped.state]
plot(result)

In [ ]:

ne_plots = []
Te_plots = []
pressures = []
for (idx,t) in enumerate(dd.core_sources.time)
    dd.global_time = t
    cp1d = dd.core_profiles.profiles_1d[]
    p1 = plot(cp1d.electrons.temperature, label = "t = $(t)", title="Te")
    push!(Te_plots,p1)
    p2 = plot(cp1d.electrons.density_thermal, label = "t = $(t)", title="ne")
    push!(ne_plots,p2)
    p3 = plot(cp1d.pressure_thermal, label = "t = $(t)", title="pressure thermal")
    push!(pressures,p3)

end
display(animate(Te_plots,"t_evolution_implemented2.gif", fps=10,ylim=(0.0,6e3)))
display(animate(ne_plots,"ne_evolution_implemented2.gif", fps=10,ylim=(0.0,1.2e20)))
display(animate(pressures,"pressure_evolution_implemented2.gif", fps=10,ylim=(0.0,1.6e5)))